In [5]:

#<<<<<<<<<<<<<<<<<<<<#          Simulacion RS        #>>>>>>>>>>>>>>>>>>>># V 1.0


import random, numpy as np, pandas as pd
from pandas import Series, DataFrame


TamañoDeLaMuestra = 500 # Cantidad de sujetos que quiero en la muestra.
QOpinionesExpuestas = 5 #Cantidad de opiniones que quiero que vea cada agente
QVecesMuestroOpinion = 5 # Cantidad de veces maxima que se muestra una opinion

In [6]:

# Defino a los agentes.

class Person (object):
    def __init__ (self, opinion,ID):
        self.Oi = opinion
        self.ID = ID # Genero un ID para cada agente
        self.IdeasVistas = [] # Me va a servir para almacenar las ideas a las que haya estado expuesto el sujeto
        self.IdeasVotadas = [] # Almaceno las ideas que voto

    
    def __str__ (self):
        try:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
        except:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
    
    def set_Oi (self, opinion):
        self.Oi = opinion
    
    def append_IdeasVistas (self, IdeaVista):
        self.IdeasVistas.append (IdeaVista)

    def append_IdeasVotadas (self, IdeaVotada):
        self.IdeasVotadas.append (IdeaVotada)           

    def votar (self, ideas):
        Comp = 1
        vota = 0
        for i in ideas.index:
            V = abs((ideas.iloc[i,1]-self.Oi)/self.Oi)
            if V == Comp:
                vota = random.randint (i, vota)
            elif V < Comp:
                vota = i
                Comp = V
        return vota # Me devuelve la id de la idea a votar     
    
    
    def get_Oi (self):
        return self.Oi
    
    def get_IdeasVistas (self):
        return self.IdeasVistas

    def get_ID (self):
        return self.ID
    
    def clean_IdeasVistas (self):
        self.IdeasVistas = []
    
    


In [7]:

# Creo la muestra
Muestra_Completa = {}
for i in range (TamañoDeLaMuestra):
    Muestra_Completa [i] = Person (np.random.uniform (-1,1),i*111)


In [35]:

#¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤ Simulacion Basica ¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤#

Muestra_Simulacion = Muestra_Completa.copy()
DataSave = pd.DataFrame(columns=['ID', 'Opinion', 'Votos', 'Vistas'])
IdeasYaMostradas = pd.DataFrame(columns=['ID', 'Opinion', 'Votos', 'Vistas'])
OpinanNoVotan = 0

for i in Muestra_Simulacion:
    if OpinanNoVotan < QOpinionesExpuestas:
        DataSave.at [OpinanNoVotan,'ID'] = Muestra_Simulacion.get(i).get_ID()
        DataSave.at [OpinanNoVotan,'Opinion'] = Muestra_Simulacion.get(i).get_Oi()
        DataSave.at [OpinanNoVotan,'Votos'] = 0
        DataSave.at [OpinanNoVotan,'Vistas'] = 0
        OpinanNoVotan +=1
    else:
        OpinionesFiltradas = DataSave[DataSave.Vistas < QVecesMuestroOpinion]
        OpinionesFiltradas.sort_values(by=['Vistas'], inplace=True)
        print (OpinionesFiltradas)
        OpinionesParaVotar = OpinionesFiltradas.iloc[:QOpinionesExpuestas, :2]
        Muestra_Simulacion.get(i).append_IdeasVistas (OpinionesParaVotar) # tendria que ajustar esto para que guarde una parte del dataframe que le paso
        DataSave.at [i,'ID'] = Muestra_Simulacion.get(i).get_ID()
        DataSave.at [i,'Opinion'] = Muestra_Simulacion.get(i).get_Oi()
        DataSave.at [i,'Votos'] = 0
        DataSave.at [i,'Vistas'] = 0
        VotoActual = Muestra_Simulacion.get(i).votar(OpinionesParaVotar)
        DataSave.at [VotoActual,'Votos'] = DataSave.at [VotoActual,'Votos'] + 1
        for i in OpinionesParaVotar.index:
             DataSave.at [i,'Vistas'] = DataSave.at [i,'Vistas'] + 1    
        
        
    
    
    
    
    

    ID   Opinion Votos Vistas
0    0  0.965361     0      0
1  111  0.622892     0      0
2  222  0.952116     0      0
3  333  0.119561     0      0
4  444  -0.76208     0      0
    ID   Opinion Votos Vistas
5  555 -0.169583     0      0
0    0  0.965361     1      1
1  111  0.622892     0      1
2  222  0.952116     0      1
3  333  0.119561     0      1
4  444  -0.76208     0      1


IndexError: single positional indexer is out-of-bounds